# Day 6: Practical Portfolio Constraints

## Week 18 - Portfolio Optimization

---

## Learning Objectives
- Understand **turnover constraints** and their role in controlling transaction costs
- Implement **cardinality constraints** to limit the number of holdings
- Apply **sector limit constraints** for diversification and compliance
- Combine multiple constraints in realistic portfolio optimization problems

## Why Constraints Matter in Practice

Real-world portfolio management requires constraints that academic mean-variance optimization often ignores:

| Constraint Type | Purpose | Challenge |
|-----------------|---------|-----------|
| **Turnover** | Control transaction costs, reduce market impact | Convex (L1 norm) |
| **Cardinality** | Limit # of positions for manageability | Non-convex (requires MIP) |
| **Sector Limits** | Regulatory compliance, diversification | Linear inequality |
| **Box Constraints** | Min/max position sizes | Linear inequality |
| **Factor Exposure** | Risk control (beta, duration) | Linear equality/inequality |

---

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")
print(f"CVXPY version: {cp.__version__}")

## 2. Generate Sample Portfolio Data

We'll create a realistic multi-asset universe with:
- 20 assets across 5 sectors
- Expected returns and covariance matrix
- Current portfolio weights (starting point for turnover)

In [ ]:
# Define asset universe
n_assets = 20
asset_names = [f"Asset_{i+1}" for i in range(n_assets)]

# Define sector assignments (5 sectors, 4 assets each)
sectors = {
    'Technology': [0, 1, 2, 3],
    'Healthcare': [4, 5, 6, 7],
    'Financials': [8, 9, 10, 11],
    'Energy': [12, 13, 14, 15],
    'Consumer': [16, 17, 18, 19]
}

# Create sector mapping for each asset
asset_sectors = {}
for sector, indices in sectors.items():
    for idx in indices:
        asset_sectors[idx] = sector

# Generate expected returns (annualized)
# Technology and Healthcare have higher expected returns
expected_returns = np.array([
    0.12, 0.14, 0.11, 0.13,  # Technology
    0.10, 0.12, 0.09, 0.11,  # Healthcare
    0.08, 0.07, 0.06, 0.08,  # Financials
    0.05, 0.04, 0.06, 0.05,  # Energy
    0.07, 0.08, 0.06, 0.09   # Consumer
])

# Generate covariance matrix using factor model approach
# This ensures positive semi-definiteness
n_factors = 5  # One factor per sector

# Factor loadings (each asset loads on its sector factor)
B = np.zeros((n_assets, n_factors))
for i, (sector, indices) in enumerate(sectors.items()):
    for idx in indices:
        B[idx, i] = 0.5 + np.random.uniform(0, 0.3)  # Loading between 0.5-0.8

# Factor covariance (sectors are somewhat correlated)
factor_cov = np.array([
    [0.04, 0.01, 0.015, 0.005, 0.01],
    [0.01, 0.035, 0.01, 0.005, 0.008],
    [0.015, 0.01, 0.03, 0.012, 0.01],
    [0.005, 0.005, 0.012, 0.05, 0.005],
    [0.01, 0.008, 0.01, 0.005, 0.025]
])

# Idiosyncratic variance
idio_var = np.diag(0.01 + 0.02 * np.random.random(n_assets))

# Full covariance matrix: Σ = B * F * B' + D
cov_matrix = B @ factor_cov @ B.T + idio_var

# Ensure symmetry
cov_matrix = (cov_matrix + cov_matrix.T) / 2

# Current portfolio weights (existing positions)
current_weights = np.array([
    0.08, 0.06, 0.04, 0.02,  # Technology: 20%
    0.05, 0.05, 0.03, 0.02,  # Healthcare: 15%
    0.08, 0.06, 0.04, 0.02,  # Financials: 20%
    0.05, 0.05, 0.05, 0.05,  # Energy: 20%
    0.08, 0.07, 0.06, 0.04   # Consumer: 25%
])
current_weights = current_weights / current_weights.sum()  # Normalize

print(f"Number of assets: {n_assets}")
print(f"Number of sectors: {len(sectors)}")
print(f"\nExpected returns range: [{expected_returns.min():.2%}, {expected_returns.max():.2%}]")
print(f"Annual volatility range: [{np.sqrt(np.diag(cov_matrix)).min():.2%}, {np.sqrt(np.diag(cov_matrix)).max():.2%}]")

In [ ]:
# Create summary DataFrame
portfolio_df = pd.DataFrame({
    'Asset': asset_names,
    'Sector': [asset_sectors[i] for i in range(n_assets)],
    'Expected_Return': expected_returns,
    'Volatility': np.sqrt(np.diag(cov_matrix)),
    'Current_Weight': current_weights
})

# Calculate current sector allocations
current_sector_weights = portfolio_df.groupby('Sector')['Current_Weight'].sum()
print("Current Sector Allocations:")
print(current_sector_weights.sort_values(ascending=False))
print(f"\nTotal: {current_sector_weights.sum():.4f}")

# Display first few rows
portfolio_df.head(10)

## 3. Turnover Constraints

### Theory

**Turnover** measures how much of the portfolio is traded:

$$\text{Turnover} = \sum_{i=1}^{n} |w_i - w_i^{\text{current}}|$$

This is the **L1-norm** of weight changes, representing total buy + sell volume.

### Why Turnover Constraints Matter

1. **Transaction Costs**: Each trade incurs commissions, bid-ask spreads
2. **Market Impact**: Large trades move prices unfavorably
3. **Tax Efficiency**: Frequent trading triggers capital gains
4. **Operational Limits**: Portfolio managers have trading capacity constraints

### Mathematical Formulation

**Hard Constraint:**
$$\sum_{i=1}^{n} |w_i - w_i^{\text{current}}| \leq \tau$$

**Soft Constraint (Penalty in Objective):**
$$\min_w \quad w^T \Sigma w - \lambda \cdot \mu^T w + \gamma \cdot \|w - w^{\text{current}}\|_1$$

where $\gamma$ is the turnover penalty parameter.

In [ ]:
def optimize_with_turnover_constraint(
    mu: np.ndarray,
    Sigma: np.ndarray,
    w_current: np.ndarray,
    target_return: float,
    max_turnover: float,
    verbose: bool = True
) -> Tuple[np.ndarray, float, float]:
    """
    Mean-variance optimization with turnover constraint.
    
    Parameters:
    -----------
    mu : Expected returns vector
    Sigma : Covariance matrix
    w_current : Current portfolio weights
    target_return : Minimum target return
    max_turnover : Maximum allowed turnover (sum of |weight changes|)
    
    Returns:
    --------
    optimal_weights, portfolio_variance, actual_turnover
    """
    n = len(mu)
    
    # Decision variable
    w = cp.Variable(n)
    
    # Objective: minimize portfolio variance
    portfolio_variance = cp.quad_form(w, Sigma)
    objective = cp.Minimize(portfolio_variance)
    
    # Constraints
    constraints = [
        cp.sum(w) == 1,                              # Fully invested
        w >= 0,                                       # Long-only
        mu @ w >= target_return,                     # Target return
        cp.norm(w - w_current, 1) <= max_turnover   # Turnover constraint (L1 norm)
    ]
    
    # Solve
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
    
    if problem.status not in ['optimal', 'optimal_inaccurate']:
        if verbose:
            print(f"Optimization failed: {problem.status}")
        return None, None, None
    
    optimal_w = w.value
    actual_turnover = np.sum(np.abs(optimal_w - w_current))
    
    if verbose:
        print(f"Status: {problem.status}")
        print(f"Portfolio Return: {mu @ optimal_w:.4%}")
        print(f"Portfolio Volatility: {np.sqrt(optimal_w @ Sigma @ optimal_w):.4%}")
        print(f"Actual Turnover: {actual_turnover:.4f} (max: {max_turnover})")
    
    return optimal_w, portfolio_variance.value, actual_turnover

In [ ]:
# Test with different turnover limits
target_return = 0.08  # 8% target return

print("=" * 60)
print("TURNOVER CONSTRAINT ANALYSIS")
print("=" * 60)

turnover_limits = [0.1, 0.2, 0.4, 0.6, 1.0, 2.0]  # 2.0 = effectively unconstrained
results_turnover = []

for max_to in turnover_limits:
    print(f"\n--- Max Turnover: {max_to:.0%} ---")
    w_opt, var_opt, actual_to = optimize_with_turnover_constraint(
        expected_returns, cov_matrix, current_weights,
        target_return=target_return, max_turnover=max_to
    )
    
    if w_opt is not None:
        results_turnover.append({
            'max_turnover': max_to,
            'actual_turnover': actual_to,
            'variance': var_opt,
            'volatility': np.sqrt(var_opt),
            'return': expected_returns @ w_opt,
            'weights': w_opt,
            'num_positions': np.sum(w_opt > 0.001)
        })

In [ ]:
# Visualize turnover constraint impact
results_df = pd.DataFrame(results_turnover)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Turnover vs Volatility Trade-off
ax1 = axes[0]
ax1.plot(results_df['max_turnover'] * 100, results_df['volatility'] * 100, 
         'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Maximum Turnover (%)', fontsize=12)
ax1.set_ylabel('Portfolio Volatility (%)', fontsize=12)
ax1.set_title('Turnover Constraint vs Risk', fontsize=14)
ax1.grid(True, alpha=0.3)

# Plot 2: Efficient Frontier with Turnover Constraint
ax2 = axes[1]
ax2.scatter(results_df['volatility'] * 100, results_df['return'] * 100, 
            c=results_df['max_turnover'] * 100, cmap='viridis', s=100, edgecolors='black')
cbar = plt.colorbar(ax2.collections[0], ax=ax2)
cbar.set_label('Max Turnover (%)')
ax2.scatter(np.sqrt(current_weights @ cov_matrix @ current_weights) * 100,
            expected_returns @ current_weights * 100, 
            c='red', s=200, marker='*', label='Current Portfolio')
ax2.set_xlabel('Volatility (%)', fontsize=12)
ax2.set_ylabel('Expected Return (%)', fontsize=12)
ax2.set_title('Risk-Return with Turnover Constraint', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Actual vs Maximum Turnover
ax3 = axes[2]
ax3.bar(range(len(results_df)), results_df['max_turnover'] * 100, 
        alpha=0.5, label='Max Allowed', width=0.4)
ax3.bar([x + 0.4 for x in range(len(results_df))], results_df['actual_turnover'] * 100, 
        alpha=0.8, label='Actual', width=0.4)
ax3.set_xlabel('Scenario', fontsize=12)
ax3.set_ylabel('Turnover (%)', fontsize=12)
ax3.set_title('Binding Turnover Constraints', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Summary: Lower turnover → Higher volatility (constraint binds)")
print("   The unconstrained solution requires ~85% turnover to reach optimal")

### Turnover Penalty (Soft Constraint) Alternative

Instead of a hard limit, we can add turnover as a penalty in the objective function:

$$\min_w \quad w^T \Sigma w + \lambda_{tc} \cdot \|w - w^{\text{current}}\|_1$$

where $\lambda_{tc}$ is the transaction cost parameter.

In [ ]:
def optimize_with_turnover_penalty(
    mu: np.ndarray,
    Sigma: np.ndarray,
    w_current: np.ndarray,
    target_return: float,
    tc_penalty: float,  # Transaction cost penalty
    risk_aversion: float = 1.0
) -> Tuple[np.ndarray, float]:
    """
    Optimization with turnover penalty in objective function.
    
    Objective: min w'Σw - λ*μ'w + γ*||w - w_current||_1
    """
    n = len(mu)
    w = cp.Variable(n)
    
    # Objective with turnover penalty
    portfolio_risk = cp.quad_form(w, Sigma)
    expected_return_term = risk_aversion * mu @ w
    turnover_penalty = tc_penalty * cp.norm(w - w_current, 1)
    
    objective = cp.Minimize(portfolio_risk - expected_return_term + turnover_penalty)
    
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        mu @ w >= target_return
    ]
    
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
    
    if problem.status in ['optimal', 'optimal_inaccurate']:
        return w.value, np.sum(np.abs(w.value - w_current))
    return None, None

# Compare hard constraint vs soft penalty
print("=" * 60)
print("HARD CONSTRAINT vs SOFT PENALTY")
print("=" * 60)

# Hard constraint: max turnover = 30%
w_hard, _, to_hard = optimize_with_turnover_constraint(
    expected_returns, cov_matrix, current_weights,
    target_return=0.08, max_turnover=0.30, verbose=False
)

# Soft penalty: find penalty that gives ~30% turnover
for tc in [0.001, 0.005, 0.01, 0.02, 0.05]:
    w_soft, to_soft = optimize_with_turnover_penalty(
        expected_returns, cov_matrix, current_weights,
        target_return=0.08, tc_penalty=tc, risk_aversion=2.0
    )
    if w_soft is not None:
        vol_soft = np.sqrt(w_soft @ cov_matrix @ w_soft)
        ret_soft = expected_returns @ w_soft
        print(f"TC Penalty={tc:.3f}: Turnover={to_soft:.2%}, Vol={vol_soft:.2%}, Ret={ret_soft:.2%}")

## 4. Cardinality Constraints

### The Challenge

**Cardinality constraint**: Limit the number of assets in the portfolio.

$$\sum_{i=1}^{n} z_i \leq K, \quad \text{where } z_i \in \{0, 1\}$$

**This makes the problem non-convex!** Requires:
- Mixed-Integer Programming (MIP) solvers
- Branch and bound algorithms
- Or heuristic approximations

### Why Cardinality Constraints Matter

1. **Operational simplicity**: Easier to manage 15 positions than 100
2. **Transaction costs**: Each position has minimum trading costs
3. **Minimum position sizes**: Small positions aren't worth holding
4. **Concentrated alpha**: If you have strong views on few stocks

### Mathematical Formulation

$$\begin{align}
\min_w \quad & w^T \Sigma w \\
\text{s.t.} \quad & \mu^T w \geq r_{\text{target}} \\
& \sum_{i=1}^n w_i = 1 \\
& w_i \geq z_i \cdot w_{\min}, \quad \forall i \\
& w_i \leq z_i \cdot w_{\max}, \quad \forall i \\
& \sum_{i=1}^n z_i \leq K \\
& z_i \in \{0, 1\}
\end{align}$$

In [ ]:
def optimize_with_cardinality_constraint(
    mu: np.ndarray,
    Sigma: np.ndarray,
    target_return: float,
    max_holdings: int,
    min_weight: float = 0.01,
    max_weight: float = 0.30,
    verbose: bool = True
) -> Tuple[np.ndarray, float]:
    """
    Mean-variance optimization with cardinality constraint (MIP).
    
    Parameters:
    -----------
    max_holdings : Maximum number of assets to hold (K)
    min_weight : Minimum weight for held assets
    max_weight : Maximum weight for any single asset
    """
    n = len(mu)
    
    # Decision variables
    w = cp.Variable(n)           # Portfolio weights
    z = cp.Variable(n, boolean=True)  # Binary: 1 if asset is held, 0 otherwise
    
    # Objective: minimize variance
    objective = cp.Minimize(cp.quad_form(w, Sigma))
    
    # Constraints
    constraints = [
        cp.sum(w) == 1,                    # Fully invested
        mu @ w >= target_return,           # Target return
        cp.sum(z) <= max_holdings,         # Cardinality constraint
        w >= z * min_weight,               # If z=1, w >= min_weight; if z=0, w >= 0
        w <= z * max_weight,               # If z=1, w <= max_weight; if z=0, w <= 0
    ]
    
    # Solve using mixed-integer solver
    problem = cp.Problem(objective, constraints)
    
    # Try ECOS_BB (branch and bound), fall back to others if not available
    try:
        problem.solve(solver=cp.ECOS_BB, mi_max_iters=1000)
    except:
        try:
            problem.solve(solver=cp.GLPK_MI)
        except:
            problem.solve(solver=cp.SCIPY)
    
    if problem.status not in ['optimal', 'optimal_inaccurate']:
        if verbose:
            print(f"Optimization failed: {problem.status}")
        return None, None
    
    optimal_w = w.value
    num_holdings = np.sum(optimal_w > 0.001)
    
    if verbose:
        print(f"Status: {problem.status}")
        print(f"Number of holdings: {num_holdings} (max: {max_holdings})")
        print(f"Portfolio Return: {mu @ optimal_w:.4%}")
        print(f"Portfolio Volatility: {np.sqrt(optimal_w @ Sigma @ optimal_w):.4%}")
        print(f"Non-zero weights: {np.sum(optimal_w > 0.001)}")
    
    return optimal_w, np.sqrt(problem.value)

In [ ]:
# Test different cardinality limits
print("=" * 60)
print("CARDINALITY CONSTRAINT ANALYSIS")
print("=" * 60)

target_return = 0.08
cardinality_limits = [5, 8, 10, 12, 15, 20]  # 20 = all assets (unconstrained)
results_cardinality = []

for max_k in cardinality_limits:
    print(f"\n--- Max Holdings: {max_k} ---")
    w_opt, vol_opt = optimize_with_cardinality_constraint(
        expected_returns, cov_matrix,
        target_return=target_return,
        max_holdings=max_k,
        min_weight=0.02,
        max_weight=0.25
    )
    
    if w_opt is not None:
        results_cardinality.append({
            'max_holdings': max_k,
            'actual_holdings': np.sum(w_opt > 0.001),
            'volatility': vol_opt,
            'return': expected_returns @ w_opt,
            'weights': w_opt,
            'max_weight': np.max(w_opt),
            'min_nonzero_weight': np.min(w_opt[w_opt > 0.001])
        })

In [ ]:
# Visualize cardinality constraint impact
card_df = pd.DataFrame(results_cardinality)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Number of holdings vs Volatility
ax1 = axes[0]
ax1.plot(card_df['max_holdings'], card_df['volatility'] * 100, 'go-', linewidth=2, markersize=8)
ax1.set_xlabel('Maximum Holdings Allowed', fontsize=12)
ax1.set_ylabel('Portfolio Volatility (%)', fontsize=12)
ax1.set_title('Cardinality Constraint vs Risk', fontsize=14)
ax1.grid(True, alpha=0.3)

# Plot 2: Portfolio Concentration
ax2 = axes[1]
if len(card_df) >= 3:
    for idx in [0, len(card_df)//2, -1]:
        row = card_df.iloc[idx]
        w = row['weights']
        nonzero_w = sorted(w[w > 0.001], reverse=True)
        ax2.plot(range(1, len(nonzero_w)+1), nonzero_w, 
                 'o-', label=f"K={row['max_holdings']}", markersize=6)
ax2.set_xlabel('Asset Rank', fontsize=12)
ax2.set_ylabel('Weight', fontsize=12)
ax2.set_title('Weight Distribution by Cardinality', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Herfindahl Index (concentration measure)
hhi = [np.sum(row['weights']**2) for _, row in card_df.iterrows()]
ax3 = axes[2]
ax3.bar(card_df['max_holdings'], hhi, color='coral', alpha=0.7)
ax3.axhline(y=1/20, color='green', linestyle='--', label='Equal-weighted HHI')
ax3.set_xlabel('Maximum Holdings', fontsize=12)
ax3.set_ylabel('Herfindahl-Hirschman Index', fontsize=12)
ax3.set_title('Portfolio Concentration', fontsize=14)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Fewer holdings → higher concentration → higher volatility")
print("   But operational simplicity may justify the trade-off")

### Heuristic Approach: Threshold-Based Selection

When MIP solvers are slow (large universes), use heuristics:

1. **Solve unconstrained** optimization
2. **Sort assets** by weight or contribution
3. **Select top K** assets and re-optimize

In [ ]:
def cardinality_heuristic(
    mu: np.ndarray,
    Sigma: np.ndarray,
    target_return: float,
    max_holdings: int
) -> np.ndarray:
    """
    Heuristic cardinality constraint:
    1. Solve unconstrained MVO
    2. Keep top K assets by weight
    3. Re-optimize with only those assets
    """
    n = len(mu)
    
    # Step 1: Solve unconstrained
    w = cp.Variable(n)
    objective = cp.Minimize(cp.quad_form(w, Sigma))
    constraints = [cp.sum(w) == 1, w >= 0, mu @ w >= target_return]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS)
    
    if prob.status not in ['optimal', 'optimal_inaccurate']:
        return None
    
    w_unconstrained = w.value
    
    # Step 2: Select top K assets
    top_k_indices = np.argsort(w_unconstrained)[-max_holdings:]
    
    # Step 3: Re-optimize with only selected assets
    w_reduced = cp.Variable(max_holdings)
    mu_reduced = mu[top_k_indices]
    Sigma_reduced = Sigma[np.ix_(top_k_indices, top_k_indices)]
    
    objective = cp.Minimize(cp.quad_form(w_reduced, Sigma_reduced))
    constraints = [
        cp.sum(w_reduced) == 1, 
        w_reduced >= 0,
        mu_reduced @ w_reduced >= target_return
    ]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS)
    
    if prob.status not in ['optimal', 'optimal_inaccurate']:
        return None
    
    # Map back to full weight vector
    w_final = np.zeros(n)
    w_final[top_k_indices] = w_reduced.value
    
    return w_final

# Compare heuristic vs MIP
print("=" * 60)
print("HEURISTIC vs MIP COMPARISON (K=8)")
print("=" * 60)

# MIP solution
w_mip, vol_mip = optimize_with_cardinality_constraint(
    expected_returns, cov_matrix, target_return=0.08, max_holdings=8, verbose=False
)

# Heuristic solution  
w_heur = cardinality_heuristic(
    expected_returns, cov_matrix, target_return=0.08, max_holdings=8
)

if w_mip is not None and w_heur is not None:
    print(f"\nMIP Solution:")
    print(f"  Volatility: {np.sqrt(w_mip @ cov_matrix @ w_mip):.4%}")
    print(f"  Return: {expected_returns @ w_mip:.4%}")
    print(f"  Holdings: {np.sum(w_mip > 0.001)}")
    
    print(f"\nHeuristic Solution:")
    print(f"  Volatility: {np.sqrt(w_heur @ cov_matrix @ w_heur):.4%}")
    print(f"  Return: {expected_returns @ w_heur:.4%}")
    print(f"  Holdings: {np.sum(w_heur > 0.001)}")
    
    print(f"\nSub-optimality: {(np.sqrt(w_heur @ cov_matrix @ w_heur) / np.sqrt(w_mip @ cov_matrix @ w_mip) - 1) * 100:.2f}%")

## 5. Sector Limit Constraints

### Theory

Sector constraints control exposure to industry groups:

$$L_s \leq \sum_{i \in S_s} w_i \leq U_s, \quad \forall s \in \text{Sectors}$$

where:
- $S_s$ = set of assets in sector $s$
- $L_s$ = lower bound for sector $s$ allocation
- $U_s$ = upper bound for sector $s$ allocation

### Types of Sector Constraints

1. **Absolute limits**: Min 5%, Max 30% in any sector
2. **Relative to benchmark**: Within ±5% of benchmark sector weights
3. **Single-sector caps**: No more than 25% in Technology
4. **Multi-level**: Industry group within sector limits

In [ ]:
def optimize_with_sector_constraints(
    mu: np.ndarray,
    Sigma: np.ndarray,
    sectors: Dict[str, List[int]],
    sector_bounds: Dict[str, Tuple[float, float]],
    target_return: float,
    verbose: bool = True
) -> np.ndarray:
    """
    Mean-variance optimization with sector exposure constraints.
    
    Parameters:
    -----------
    sectors : Dict mapping sector name to list of asset indices
    sector_bounds : Dict mapping sector name to (lower_bound, upper_bound)
    """
    n = len(mu)
    w = cp.Variable(n)
    
    # Objective: minimize variance
    objective = cp.Minimize(cp.quad_form(w, Sigma))
    
    # Base constraints
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        mu @ w >= target_return
    ]
    
    # Sector constraints
    for sector_name, indices in sectors.items():
        lower, upper = sector_bounds.get(sector_name, (0, 1))
        sector_weight = cp.sum(w[indices])
        constraints.append(sector_weight >= lower)
        constraints.append(sector_weight <= upper)
    
    # Solve
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS)
    
    if problem.status not in ['optimal', 'optimal_inaccurate']:
        if verbose:
            print(f"Optimization failed: {problem.status}")
        return None
    
    optimal_w = w.value
    
    if verbose:
        print(f"Status: {problem.status}")
        print(f"Portfolio Return: {mu @ optimal_w:.4%}")
        print(f"Portfolio Volatility: {np.sqrt(optimal_w @ Sigma @ optimal_w):.4%}")
        print("\nSector Allocations:")
        for sector_name, indices in sectors.items():
            actual = np.sum(optimal_w[indices])
            lower, upper = sector_bounds.get(sector_name, (0, 1))
            binding = "⚠️" if (abs(actual - lower) < 0.001 or abs(actual - upper) < 0.001) else ""
            print(f"  {sector_name}: {actual:.2%} [{lower:.0%}-{upper:.0%}] {binding}")
    
    return optimal_w

In [ ]:
# Test sector constraints
print("=" * 60)
print("SECTOR CONSTRAINT ANALYSIS")
print("=" * 60)

# Scenario 1: No sector constraints (unconstrained)
print("\n--- Scenario 1: No Sector Constraints ---")
sector_bounds_none = {s: (0.0, 1.0) for s in sectors}
w_no_sector = optimize_with_sector_constraints(
    expected_returns, cov_matrix, sectors, 
    sector_bounds_none, target_return=0.08
)

# Scenario 2: Balanced sector constraints (10-30% each)
print("\n--- Scenario 2: Balanced Constraints (10-30% each) ---")
sector_bounds_balanced = {s: (0.10, 0.30) for s in sectors}
w_balanced = optimize_with_sector_constraints(
    expected_returns, cov_matrix, sectors,
    sector_bounds_balanced, target_return=0.08
)

# Scenario 3: Strict constraints (15-25% each)
print("\n--- Scenario 3: Strict Constraints (15-25% each) ---")
sector_bounds_strict = {s: (0.15, 0.25) for s in sectors}
w_strict = optimize_with_sector_constraints(
    expected_returns, cov_matrix, sectors,
    sector_bounds_strict, target_return=0.08
)

In [ ]:
# Visualize sector allocations across scenarios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calculate sector allocations for each scenario
sector_names = list(sectors.keys())
scenarios = ['No Constraints', 'Balanced (10-30%)', 'Strict (15-25%)']
weights_list = [w_no_sector, w_balanced, w_strict]

allocations = []
for w in weights_list:
    if w is not None:
        sector_alloc = [np.sum(w[sectors[s]]) for s in sector_names]
    else:
        sector_alloc = [0] * len(sector_names)
    allocations.append(sector_alloc)

# Plot 1: Grouped bar chart of sector allocations
x = np.arange(len(sector_names))
width = 0.25

for i, (scenario, alloc) in enumerate(zip(scenarios, allocations)):
    axes[0].bar(x + i*width, [a*100 for a in alloc], width, label=scenario, alpha=0.8)

axes[0].set_xlabel('Sector', fontsize=12)
axes[0].set_ylabel('Allocation (%)', fontsize=12)
axes[0].set_title('Sector Allocations by Constraint Scenario', fontsize=14)
axes[0].set_xticks(x + width)
axes[0].set_xticklabels(sector_names, rotation=45, ha='right')
axes[0].legend()
axes[0].axhline(y=20, color='gray', linestyle='--', alpha=0.5, label='Equal weight')
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: Risk-return comparison
vol_list = []
ret_list = []
for w in weights_list:
    if w is not None:
        vol_list.append(np.sqrt(w @ cov_matrix @ w) * 100)
        ret_list.append(expected_returns @ w * 100)
    else:
        vol_list.append(np.nan)
        ret_list.append(np.nan)

colors = ['blue', 'green', 'red']
for i, (scenario, vol, ret) in enumerate(zip(scenarios, vol_list, ret_list)):
    axes[1].scatter(vol, ret, s=150, c=colors[i], label=scenario, edgecolors='black')
    axes[1].annotate(scenario, (vol, ret), xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[1].set_xlabel('Volatility (%)', fontsize=12)
axes[1].set_ylabel('Expected Return (%)', fontsize=12)
axes[1].set_title('Risk-Return Trade-off with Sector Constraints', fontsize=14)
axes[1].legend(loc='lower right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Insight: Tighter sector constraints → more diversified but higher volatility")

### Benchmark-Relative Sector Constraints

In practice, active managers often constrain sector deviations from benchmark:

$$|w_s^{\text{portfolio}} - w_s^{\text{benchmark}}| \leq \delta_s$$

This is the **tracking error constraint** at sector level.